# **Classificazione di immagini tramite Multi-Layer Perceptron (MLP)**

In questa esercitazione impariamo a risolvere un problema di classificazione utilizzando una semplice rete neurale MLP.

I dati di input sono immagini, quindi sono dati ad **alta dimensionalità**, presi dal dataset Euclid (lo trovate su Virtuale).

La rete impara (*learn*) **da sola** a capire come risolvere il problema.

### Librerie
Importiamo qui le librerie necessarie alla soluzione:

*   `Scikit-Learn (sklearn)`: libreria molto potente e vasta principalmente creata per il **Machine Learning** (con alcuni elementi di Deep Learning, come le reti MLP).
*   `OpenCV (cv2)`: libreria utilizzata principalmente per gestire le immagini a livello di codice.


In [5]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from glob import glob
from os.path import join
from tqdm import tqdm
import cv2

### Funzioni di supporto
Qui definiamo le funzioni che utilizzeremo nella nostra soluzione.

Le funzioni sono utili per avere un codice più modulare e ordinato.

`get_labels()` è una funzione che riceve il nome del dato (`string`) e restituisce la classe - **etichetta** -  (`int`), seguendo questa convenzione:

*   Triangolo: 0
*   Rettangolo: 1
*   Quadrato: 2
*   Rombo: 3

Esempio: 0_triangle.png → 0

In [6]:
def get_labels(name):
    if 'triangle' in name:
        return 0
    elif 'square' in name:
        return 1
    elif 'rectangle' in name:
        return 2
    elif 'rhombus' in name:
        return 3
    else:
        raise NotImplementedError('Not existing class!')

`extract_feature()` è una funzione che, data una lista di immagini, calcola le feature associate. Per noi oggi, queste feature corrispondono al contenuto delle immagini "srotolato" con valori compresi tra 0 e 1.

Per aprire le immagini utilizziamo la libreria `opencv`. Il contenuto delle immagini viene normalizzato tra 0 e 1.

**Tools**:
*   `cv2.imread()`: apre un'immagine (0: livelli di grigio, 1: colori)
*   `cv2.resize()`: ridimensiona un'immagine


In [7]:
def extract_features(images, feat_type, img_size):

    labels = []
    features = []

    for image in tqdm(images):

        # open the image
        img = cv2.imread(image, 0)

        # resize the image
        img = cv2.resize(img, (img_size, img_size))

        # compute the features
        if feat_type == 'img':
            img = img / 255.0
            feat = np.ravel(img)
        else:
            raise NotImplementedError('Not implemented feature!')

        # append features and labels
        features.append(feat)
        labels.append(get_labels(image))

    return features, labels

### Soluzione
Carica il file .zip che contiene il dataset *Euclid*.
Estrai il contenuto con il comando seguente:

!unzip -q Euclid_dataset.zip -d ./content

In [9]:
dataset_path = './content/Euclid_dataset'
images = glob(join(dataset_path, '*', '*.png'))
print('Images: ', len(images))

Images:  800


Ora è fondamentale dividere il nostro dataset in train, validation e test.
Prima di fare questo è necessario mescolare tutti i dati.

**Tools**:
-    `np.random.shuffle()`: modifica una sequenza mescolando gli elementi.

In [10]:
print('Prima shuffling: {}'.format(images[:10]))
np.random.shuffle(images)
print('Dopo shuffling: {}'.format(images[:10]))

Prima shuffling: ['./content/Euclid_dataset/square/60_square.png', './content/Euclid_dataset/square/43_square.png', './content/Euclid_dataset/square/89_square.png', './content/Euclid_dataset/square/138_square.png', './content/Euclid_dataset/square/70_square.png', './content/Euclid_dataset/square/93_square.png', './content/Euclid_dataset/square/108_square.png', './content/Euclid_dataset/square/190_square.png', './content/Euclid_dataset/square/167_square.png', './content/Euclid_dataset/square/162_square.png']
Dopo shuffling: ['./content/Euclid_dataset/square/117_square.png', './content/Euclid_dataset/rhombus/37_rhombus.png', './content/Euclid_dataset/triangle/9_triangle.png', './content/Euclid_dataset/square/100_square.png', './content/Euclid_dataset/rectangle/125_rectangle.png', './content/Euclid_dataset/rectangle/77_rectangle.png', './content/Euclid_dataset/rhombus/19_rhombus.png', './content/Euclid_dataset/square/17_square.png', './content/Euclid_dataset/rectangle/101_rectangle.png', 

Mettiamo il **20% dei dati in training, 20% in validation**, e la rimanente parte **(60%) nel test set**.

In [11]:
trainset = images[:int(0.2*len(images))]
valset = images[int(0.2*len(images)):int(0.4*len(images))]
testset = images[int(0.4*len(images)):]
print('Total: {} splitted in Train: {}, Val: {} and Test: {}'.format(len(images), len(trainset), len(valset), len(testset)))

Total: 800 splitted in Train: 160, Val: 160 and Test: 480


Da questo momento, avremo **tre insiemi di dati**: train, validation e test.

Ogni elemento del dataset può appartenere solo a uno di questi, gli insieme quindi sono completamente **disgiunti**.

In [12]:
img_size = 32
feature_type = 'img'

train_x, train_y = extract_features(trainset, feature_type, img_size)
val_x, val_y = extract_features(valset, feature_type, img_size)
test_x, test_y = extract_features(testset, feature_type, img_size)

print('Train: {}, Val: {} and Test: {}'.format(len(train_x), len(val_x), len(test_x)))
print('Total: {}'.format(len(train_x) + len(val_x) + len(test_x)))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 480/480 [00:00<00:00, 1276.04it/s]

Train: 160, Val: 160 and Test: 480
Total: 800


### Classificatore
Ora definiamo il nostro classificatore che, come detto, è un MLP.
Per ora utilizziamo la configurazione di base.

In [15]:
clf = MLPClassifier()

### Training
Ora siamo pronti ad addestare il nostro MLP!

Fortunatamente la complessità del training di una rete neurale è interamente gestita da `sklearn` (che è molto semplice da utilizzare)

**Tools**:
-   `model.fit()`: addestra il modello

In [16]:
clf.fit(train_x, train_y)

/home/pinacoteca/Unibo/Numerical_Methods_for_AI/.venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier()

### Validazione

E' ora di validare il modello per trovare i parametri più corretti.

Possiamo cambiare i parametri o i dati del modello, poi eseguire nuovamente un addestramento, e infine capire se le modifiche hanno portato a un incremento o decremento delle prestazioni.

**Tools:**
*   `score()`: validazione del modello.



In [17]:
print('Validation accuracy: {:.3f}'.format(clf.score(val_x, val_y)))

Validation accuracy: 0.381


### Test
Finalmente possiamo testare il nostro modello, facendogli predire in output le classi!

**Tools**:
  - `model.predict()`: predice la classe per un dato di input

In [20]:
pred_y = clf.predict(test_x)
print('Predicted {} samples: {}'.format(len(pred_y), pred_y))
print('GT {} samples: {}'.format(len(test_y), test_y))
#print(pred_y==test_y)

Predicted 480 samples: [1 0 0 3 1 2 3 0 2 1 1 3 3 0 0 2 0 0 2 3 2 3 2 2 1 0 0 1 0 0 3 3 3 0 1 3 3
 2 0 0 3 3 2 1 1 0 2 1 3 3 3 3 0 2 0 3 0 3 0 3 0 3 1 3 0 3 0 3 0 0 2 1 3 1
 1 2 3 0 3 1 0 2 3 1 0 0 3 3 1 0 1 1 1 1 1 0 1 2 0 0 3 3 0 3 1 3 1 1 3 1 1
 3 0 1 3 2 2 3 0 1 3 3 3 3 1 1 0 3 3 2 0 0 0 3 1 0 0 1 0 2 0 1 1 0 1 2 0 0
 2 0 2 1 0 2 3 2 1 0 0 2 0 0 0 3 3 0 1 3 3 1 0 0 1 0 0 1 0 0 3 2 2 0 0 1 3
 3 3 1 0 1 3 1 1 1 0 1 3 3 1 1 0 0 2 1 0 1 1 0 3 3 0 1 3 3 0 0 0 0 2 1 2 1
 3 2 0 1 3 3 2 1 1 3 0 1 3 1 3 1 3 1 1 3 0 1 2 1 0 2 2 3 0 3 2 1 0 3 0 1 0
 3 3 0 1 3 0 3 1 0 0 1 2 0 1 3 3 1 0 2 3 2 1 3 1 3 3 1 1 1 0 3 3 0 1 3 1 1
 3 3 0 2 1 2 1 0 3 1 1 3 0 1 0 3 0 0 0 0 2 1 1 2 2 0 0 1 1 3 3 0 3 0 3 3 3
 0 0 0 2 0 1 0 2 1 2 0 3 3 3 0 3 0 3 0 1 3 1 1 3 1 3 3 0 0 1 3 2 0 0 3 0 3
 1 3 2 1 3 0 1 0 3 1 3 1 0 0 1 3 1 0 0 0 0 1 0 0 0 3 1 0 0 3 1 3 0 0 3 2 3
 0 3 3 3 0 3 3 0 0 1 1 1 2 3 2 1 3 0 3 1 3 3 1 1 3 2 1 0 0 3 0 3 3 1 3 0 1
 0 0 3 1 0 0 2 0 1 2 0 0 0 1 3 3 2 3 1 3 1 0 1 3 3 0 0 0 2 2 0 0 1 3 0 3]
GT 


**Tools**:
   * `accuracy_score()`: score di accuratezza della classificazione, date in input le classi predette

In [21]:
print('Final Accuracy: {:.3f}'.format(accuracy_score(test_y, pred_y)))

Final Accuracy: 0.438


### Conclusioni
La soluzione implementata purtroppo raggiunge una **accuratezza relativamente bassa** (tieni conto che una soluzione random raggiungerebbe il 25% circa di accuratezza).

Questo principalmente non è dovuto solamente al classificatore, ma all'alta dimensionalità dei dati in input.

Ci sono quindi tre possibili soluzioni:


*   **Modificare i parametri del classificatore**, per guadagnare qualcosa in termini di prestazioni. Documentazione: https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
*   **Condensare il contenuto informativo dei dati in input**, utilizzando i cosidetti *feature descriptor* (per esempio gli HOG https://en.wikipedia.org/wiki/Histogram_of_oriented_gradients#:~:text=The%20histogram%20of%20oriented%20gradients,localized%20portions%20of%20an%20image). Questo è uno strumento di Visione Artificiale avanzato che non verrà trattato in questo corso.
*   **Utilizzare una Convolutional Neural Network (CNN)** che, come visto a lezione, è uno strumento ottimizzato per le immagini. In particolare, è in grado di estrarre in maniera autonoma le feature dalle immagini (nella parte di *Feature Extractor*).

E' importante ricordare che, in presenza di dati a bassa dimensionalità (come i valoro numerici provenienti per esempio da un sistema di domotica), una rete MLP dovrebbe avere ottime performance.

Nella prossima esercitazione vedremo come implementare una CNN.